# Playing with Sqlite3

In [2]:
import sqlite3 as sql


#start with a connection object
conn = sql.connect('example.db')

# create a cursor object and use its execute method to run SQL commands
c = conn.cursor()

In [1]:
#execute a sql command
c.execute('''CREATE TABLE names (name text, dob text, salary real, favcolor text)
''')

OperationalError: table names already exists

In [3]:
#insert into table
c.execute(""" INSERT INTO names VALUES ('harrison', '4/25/1995', 100000, 'blue')""")

# Commit the changes
conn.commit()

#close the connection
conn.close()

In [4]:
#reopen the connection to see if changes are still there
conn2 = sql.connect("example.db")
c = conn2.cursor()

You should not assemble your query using python's string operations. Use parameter substitution instead

Put ? as a placeholder wherever you want to  use a value, and then provide a tuple of values in the .execute() method's seconf argument,

In [7]:
#use a python tuple object and insert into a SQL query
names = ("harrison",)
c.execute("SELECT * FROM names WHERE name=?", names)
c.fetchall()

[('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue')]

In [8]:
#inserting many examples with parameter substitution
inserts = [('huckle', '11/11/2016', 0, 'red'),
('chicken', '629836', 182098, 'green')]

#insert into the db with the cursor object
c.executemany("INSERT INTO names VALUES (?,?,?,?)", inserts)

conn2.commit()

c.execute("SELECT * FROM names WHERE name='huckle'")
print(c.fetchone())

('huckle', '11/11/2016', 0.0, 'red')


In [9]:
for row in c.execute("SELECT * FROM names ORDER BY salary"):
    print(row)

('huckle', '11/11/2016', 0.0, 'red')
('harrison', '4/25/1995', 100000.0, 'blue')
('harrison', '4/25/1995', 100000.0, 'blue')
('harrison', '4/25/1995', 100000.0, 'blue')
('harrison', '4/25/1995', 100000.0, 'blue')
('chicken', '629836', 182098.0, 'green')


In [10]:
conn2.close()

# SQL vs Pandas

Data is not synonymous with relational database. (Tables that look like excel files). We can have CSV files, plain text,  Parquet, and more.

Pandas is particularly geared towards structured data

### The anatomy of a SQL query

SELECT ... FROM ... WHERE

GROUP BY ... HAVING

ORDER BY ...

In [11]:
conn = sql.connect("example.db")
c = conn.cursor()

In [12]:
c.execute("SELECT * FROM names WHERE name='harrison' OR favcolor='green'")
c.fetchall()

[('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('chicken', '629836', 182098.0, 'green')]

In [14]:
c.execute("SELECT * FROM names WHERE favcolor IN ('green', 'red')")
c.fetchall()

[('huckle', '11/11/2016', 0.0, 'red'),
 ('chicken', '629836', 182098.0, 'green')]

In [15]:
c.execute("SELECT * FROM names GROUP BY name")
c.fetchall()

[('chicken', '629836', 182098.0, 'green'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('huckle', '11/11/2016', 0.0, 'red')]

In [19]:
c.execute("SELECT * FROM names GROUP BY name ORDER BY salary DESC")
c.fetchall()

[('chicken', '629836', 182098.0, 'green'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('huckle', '11/11/2016', 0.0, 'red')]

In [21]:
c.execute("SELECT *, COUNT(*) FROM names")
c.fetchall()

[('chicken', '629836', 182098.0, 'green', 6)]

### Aggregate Functions

In [25]:
c.execute("SELECT MAX(salary) FROM names")
c.fetchall()
conn.close()

### SQL Into Pandas

In [31]:
import pandas as pd
conn = sql.connect('example.db')

df = pd.read_sql('select * from names', conn)

df

,name,dob,salary,favcolor
0,harrison,4/25/1995,100000.0,blue
1,harrison,4/25/1995,100000.0,blue
2,harrison,4/25/1995,100000.0,blue
3,harrison,4/25/1995,100000.0,blue
4,huckle,11/11/2016,0.0,red
5,chicken,629836,182098.0,green
